# Imports

In [2]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import penaltyblog as pb

from penaltyblog.scrapers import FootballData
from penaltyblog.matchflow import Flow, where_equals, get_field

import warnings
from statsbombpy.api_client import NoAuthWarning
warnings.filterwarnings("ignore", category=NoAuthWarning)

## 2.1: Fetch Match Data and Pinnacle Odds

In [5]:
LEAGUE = "ENG Premier League"
SEASON = "2015-2016" 

print(f"Fetching match data and Pinnacle odds for {LEAGUE} {SEASON}...")

df_odds = FootballData(LEAGUE, SEASON).get_fixtures()

odds_cols = [
    "date",
    "team_home",
    "team_away",
    "goals_home",
    "goals_away",
    "psh", 
    "psd", 
    "psa", 
]

df_v4 = df_odds[odds_cols].copy()
df_v4.dropna(subset=['psh', 'psd', 'psa'], inplace=True)
df_v4.sort_values(by='date', inplace=True)
df_v4.reset_index(drop=True, inplace=True)

print(f"\nSuccessfully fetched {len(df_v4)} matches with Pinnacle odds.")
display(df_v4.head())

Fetching match data and Pinnacle odds for ENG Premier League 2015-2016...

Successfully fetched 380 matches with Pinnacle odds.


,date,team_home,team_away,goals_home,goals_away,psh,psd,psa
0,2015-08-08,Bournemouth,Aston Villa,0,1,1.95,3.65,4.27
1,2015-08-08,Chelsea,Swansea,2,2,1.39,4.92,10.39
2,2015-08-08,Everton,Watford,2,2,1.70,3.95,5.62
3,2015-08-08,Leicester,Sunderland,4,2,1.99,3.48,4.34
4,2015-08-08,Man United,Tottenham,1,0,1.65,4.09,5.90


## Step 2.2: Combine Odds Data with StatsBomb Match IDs

In [8]:
COMPETITION_ID = 2
SEASON_ID = 27

print("Fetching official match list from StatsBomb...")
sb_matches_raw = (
    Flow.statsbomb.matches(competition_id=COMPETITION_ID, season_id=SEASON_ID)
    .select("match_id", "match_date", "home_team", "away_team")
    .collect()
)
df_sb_matches = pd.DataFrame(sb_matches_raw)

df_sb_matches['team_home'] = df_sb_matches['home_team'].apply(lambda x: x['home_team_name'])
df_sb_matches['team_away'] = df_sb_matches['away_team'].apply(lambda x: x['away_team_name'])
df_sb_matches.rename(columns={'match_date': 'date'}, inplace=True)

df_sb_matches = df_sb_matches[['match_id', 'date', 'team_home', 'team_away']]

df_v4['date'] = pd.to_datetime(df_v4['date'])
df_sb_matches['date'] = pd.to_datetime(df_sb_matches['date'])

print("Merging odds data with StatsBomb match IDs...")
df_master = pd.merge(
    df_v4,
    df_sb_matches,
    on=['date', 'team_home', 'team_away'],
    how='inner'
)

print(f"\nSuccessfully merged {len(df_master)} matches.")
print("We now have the Pinnacle odds and the StatsBomb match_id in one place.")
display(df_master.head())

Fetching official match list from StatsBomb...
Merging odds data with StatsBomb match IDs...

Successfully merged 72 matches.
We now have the Pinnacle odds and the StatsBomb match_id in one place.


,date,team_home,team_away,goals_home,goals_away,psh,psd,psa,match_id
0,2015-08-08,Everton,Watford,2,2,1.70,3.95,5.62,3754300
1,2015-08-15,Southampton,Everton,0,3,1.98,3.60,4.21,3754034
2,2015-08-16,Crystal Palace,Arsenal,1,2,5.34,3.97,1.72,3754312
3,2015-08-22,Crystal Palace,Aston Villa,2,1,1.85,3.63,4.89,3754190
4,2015-08-23,Watford,Southampton,0,0,2.76,3.36,2.78,3754189
